# Simultaneous global and target analysis of the timeresolved fluorescence of the megacomplex at 77K

### Defining datasets and inspect data

In [ ]:
from __future__ import annotations

from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.inspect import show_a_matrixes
from pyglotaran_extras import (
    plot_overview,
    plot_data_overview,
    plot_fitted_traces,
    select_plot_wavelengths,
)

The code below defined the (groups of) datasets used in the analysis. Only for a single dataset the plot_data_overview is shown to avoid repetition, it is left as an exercise to the reader to inspect the other data.

In [ ]:
DATA_PATH1 = "data/supercomplex_targeta.ascii"
DATA_PATH2 = "data/supercomplex_targetb.ascii"

plot_data_overview(
    DATA_PATH1,
    nr_of_data_svd_vectors=4,
    irf_location=470,
    title="MCL at 77K exc 610 nm",
    # vmax=60
);

# Create target scheme and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/20241110streak_target_77K_supercomplex.yml",  
    parameters="models/20241120streak_target_77K_supercomplex.csv",
    maximum_number_function_evaluations=11,
    clp_link_tolerance=0.1,
    data={
        "super1ns": DATA_PATH1,
        "super2ns": DATA_PATH2,
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result1 = optimize(target_scheme, raise_exception=True)

For reference, the final Cost should be
- 11         1.7094e+05


In [ ]:
target_result1

## Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas,plot_norm_sas
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.style import ColorCode
# ColorCode.green,"k",
myFRLcolors = [ "b","k", "r",ColorCode.magenta,ColorCode.indigo,ColorCode.green,"g",ColorCode.turquoise,ColorCode.grey]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
myFRLcolors2 = [ ColorCode.cyan,"g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.indigo, ColorCode.brown, "w","w","w","y",ColorCode.maroon]
res=target_result1.data["super1ns"]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 9)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(target_result1.data["super1ns"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
# plot_concentrations(target_result1.data["super2ns"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_concentrations(target_result1.data["super2ns"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler,linetype="--")
# plot_concentrations(target_result1.data["WLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
# custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result1.data["FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_concentrations(target_result1.data["FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_sas(target_result1.data["FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result1.data["super1ns"], axes[1], cycler=custom_cycler)
selPS=["PSI1","PSI2","PSI3","PSII1","PSII2","PSII3"]
myFRLcolorsPS = [ "r",ColorCode.magenta,ColorCode.indigo,ColorCode.green,"g",ColorCode.turquoise,ColorCode.grey]
custom_cyclerPS = cycler(color=myFRLcolorsPS)
plot_norm_sas(target_result1.data["super1ns"], axes[2], cycler=custom_cyclerPS,selected_species=selPS)
# plot_das(target_result1.data["super1ns"], axes[2], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

## Create the WL guidance data sets

In [ ]:
from glotaran.io import load_dataset, load_model, load_parameters
from glotaran.io import save_dataset,load_dataset
from glotaran.utils.io import create_clp_guide_dataset

for species in target_result1.data["super1ns"].species:
    clp_guide = create_clp_guide_dataset(target_result1.data["super1ns"], species.item())
    string_in_string = "guide/20241120megacomplex77K_{}.ascii".format(species.item())
    save_dataset(clp_guide.data, string_in_string,allow_overwrite=True)

In [ ]:
datasetPSI1 = load_dataset(
    "guide/20241120megacomplex77K_PSI1.ascii"
)
datasetPSII1 = load_dataset(
    "guide/20241120megacomplex77K_PSII1.ascii"
)

In [ ]:
spectral_model = load_model("models/spectral_model.yml")
spectral_parameters = load_parameters("models/spectral_params.yml")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": datasetPSI1},
    maximum_number_function_evaluations=25,
)
spectral_result = optimize(spectral_scheme)
spectral_result.optimized_parameters

In [ ]:
spectral_result

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_overview

fig, axes = plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
from glotaran.io import save_dataset

save_dataset(
    spectral_result.data["dataset"].fitted_data,
    "guide/20241120megacomplex77K_PSI1_fitted.ascii",
    allow_overwrite=True,
)

In [ ]:
spectral_model = load_model("models/spectral_model2.yml")
spectral_parameters = load_parameters("models/spectral_params2.yml")
spectral_model.validate(parameters=spectral_parameters)

spectral_scheme = Scheme(
    spectral_model,
    spectral_parameters,
    data={"dataset": datasetPSII1},
    maximum_number_function_evaluations=25,
)
spectral_result = optimize(spectral_scheme)
spectral_result.optimized_parameters

In [ ]:
spectral_result

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_overview

fig, axes = plot_overview(spectral_result.data["dataset"], linlog=False)

In [ ]:
from glotaran.io import save_dataset

save_dataset(
    spectral_result.data["dataset"].fitted_data,
    "guide/20241120megacomplex77K_PSII1_fitted.ascii",
    allow_overwrite=True,
)

In [ ]:
spectral_result.optimized_parameters

# refined target

In [ ]:
DATA_PATHPSII1="guide/20241120megacomplex77K_PSII1_fitted.ascii"
DATA_PATHPSI1="guide/20241120megacomplex77K_PSI1_fitted.ascii"
target_scheme = Scheme(
    model="models/20250201streak_target_77K_supercomplex.yml",  # type: ignore
    parameters="models/20241120streak_target_77K_supercomplex.csv",
    maximum_number_function_evaluations=11,
    clp_link_tolerance=0.1,
    data={
        "super1ns": DATA_PATH1,
        "super2ns": DATA_PATH2,
        "dataPSI1": DATA_PATHPSI1,
        "dataPSII1": DATA_PATHPSII1,
    },  # type: ignore
)
target_scheme.validate()

In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result1 = optimize(target_scheme, raise_exception=True)

For reference, the final Cost should be
- 11         1.7109e+05


In [ ]:
target_result1

# Fig.5 Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas,plot_norm_sas
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.style import ColorCode
# ColorCode.green,"k",
myFRLcolors = [ "b","k", "r",ColorCode.magenta,ColorCode.indigo,ColorCode.green,"g",ColorCode.turquoise,ColorCode.grey]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
myFRLcolors2 = [ ColorCode.cyan,"g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.indigo, ColorCode.brown, "w","w","w","y",ColorCode.maroon]
res=target_result1.data["super1ns"]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
custom_cycler2 = cycler(color=myFRLcolors, linestyle=["--"] * 9)
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(target_result1.data["super1ns"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
# plot_concentrations(target_result1.data["super2ns"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_concentrations(target_result1.data["super2ns"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler,linetype="--")
# plot_concentrations(target_result1.data["WLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
# custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result1.data["FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_concentrations(target_result1.data["FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# plot_sas(target_result1.data["FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result1.data["super1ns"], axes[1], cycler=custom_cycler)
selPS=["PSI1","PSI2","PSI3","PSII1","PSII2","PSII3"]
myFRLcolorsPS = [ "r",ColorCode.magenta,ColorCode.indigo,ColorCode.green,"g",ColorCode.turquoise,ColorCode.grey]
custom_cyclerPS = cycler(color=myFRLcolorsPS)
plot_norm_sas(target_result1.data["super1ns"], axes[2], cycler=custom_cyclerPS,selected_species=selPS)
# plot_das(target_result1.data["super1ns"], axes[2], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)

In [ ]:
res.lifetime_supercomplex1

## Print the estimates of the optimized parameters and their precision 
The t-values of the **free** parameters (Vary=True) indicate the precision

In [ ]:
target_result1.optimized_parameters

## Fit quality of the target analysis of the WL data
overlays of traces and fits, first of 16 wavelengths, then of 6 selected wavelengths

In [ ]:
target_result_streak = (
    # target_result.data["FRLtr1"],
    # target_result1.data["super1ns"],
    target_result1,
    # target_result.data["FRLtr2"],
    # target_result.data["WLtr2"],
    # # target_result.data["FRLtr4"],
    # target_result.data["super1ns"],
)

In [ ]:
import numpy as np
# wavelengths = select_plot_wavelengths(
#     target_result_streak, equidistant_wavelengths=True
# )
wavelengths =np.linspace(645, 750, num=16)
plot_fitted_traces(target_result1, wavelengths, linlog=True, linthresh=1000);

# overview 1 ns

In [ ]:
fig, axes = plot_overview(target_result1.data["super1ns"], linlog=True, linthresh=100)


In [ ]:
fig, axes = plot_overview(target_result1.data["super2ns"], linlog=True, linthresh=100)


In [ ]:
# save_result(
#     result=target_result1,
#     result_path="resultssupercomplex/result.yaml",
#     allow_overwrite=True,
# )

# Fig.S4 selected traces & fits

In [ ]:
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        target_result1,
        [685,705,715,725,740,750],
        linlog=True,
        linthresh=1000,
        axes_shape=(3, 2),
        figsize=(8,6),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.cyan,
                cc.blue,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "1 ns"
        elif i == 3:
            labels[i] = "2 ns"
        elif i == 5:
            labels[i] = "TR4"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 3:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            # line.set_linewidth(0.5)  # Set the line width here
            line.set_linewidth(1)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()

### Compute the FWHM of the main Gaussian of the IRF

In [ ]:
import numpy as np
const=2*np.sqrt(2*np.log(2))
[
const*target_result1.optimized_parameters.get("super1nsirf.width1").value,
const*target_result1.optimized_parameters.get("super1nsirf.width2").value,
const*target_result1.optimized_parameters.get("super1nsirf.scale2").value,
const*target_result1.optimized_parameters.get("super2nsirf.width1").value,
const*target_result1.optimized_parameters.get("super2nsirf.width2").value,
const*target_result1.optimized_parameters.get("super2nsirf.scale2").value,
]